In [1]:
#naive calculation
nb = 200000 * 32
kb = 16 * 32
nl2k = 0.005 * 200000 * np.log2(16) + 200000/16 * np.log2(16)
(nb/(kb + nl2k))

NameError: name 'np' is not defined

In [2]:
import numpy as np

number_of_elements = 266610
pruning_rate = 0.043/2
index_bits = 5
#number_of_elements = 400000
#pruning_rate = 0.005/2
#index_bits = 6

orig = number_of_elements * 32 #266k elements at 32 bit precision
codebook = 16 * 32 # 16 cluster means at 32 bit preicison
weight = number_of_elements * np.log2(32) * (pruning_rate + 1/(2**index_bits))
index = number_of_elements * index_bits * (pruning_rate + 1/(2**index_bits))
(orig/(codebook + weight + index))



60.443457621222713

In [5]:
#encode
R = np.random.rand(400000)
R[R<=0.995] = 0
R[R>0.995] = 1/100
'''
R[R>0.990] = 2/100
R[R>0.985] = 3/100
R[R>0.980] = 4/100
R[R>0.975] = 5/100
R[R>0.970] = 6/100
R[R>0.965] = 7/100
R[R>0.960] = 8/100
R[R>0.959] = 9/100
R[R>0.957] = 10/100
'''

R=R*100
#[[x,list(R).count(x)] for x in set(list(R))]

model =

#code_list
class compressed_model():
    def __init__(state_dict, gmp_list):
        #could be multiple gmps
        #could be multiple gmp scales
        self.means = []
        for g in gmp_list:
            if (g.scaling):
                for 
            else:
                self.means.append(list[g.means.clone().data.cpu().numpy()])
        

    def encode(R, index_bits):
        index_spacing = index_bits**2
        index_list = []
        weight_list = []
        index_counter=0
        for pos, weight in enumerate(list(R)):
            index_counter+=1
            if (weight==0):
                if index_counter==index_spacing:
                    index_list.append(index_spacing)
                    weight_list.append(0)
                    index_counter=0
            else:
                weight_list.append(weight)
                index_list.append(index_counter)
                index_counter=0
        if(index_counter>0):
            index_list.append(index_counter)
            weight_list.append(0)
        return index_list, weight_list

index_size = len(index_list) * index_bits
weight_size = len(weight_list) * 4
codebook_size = 32 * 2**4
full_size = len(list(R)) * 32
print(full_size, len(weight_list), index_size, weight_size, codebook_size, full_size/(index_size + weight_size + codebook_size))

12800000 6017 54153 24068 512 162.57477804732451


In [6]:
#decode
R_recov = []
for index, weight in zip(index_list, weight_list):
    for z in range(int(index-1)):
        R_recov.append(0)
    R_recov.append(weight)

In [7]:
(R - R_recov).sum()

0.0